In [1]:
!pip install datasets
!pip install pytorch_lightning
!pip install py-readability-metrics
!python -m nltk.downloader punkt
!pip install evaluate
!pip install sacremoses sacrebleu
!pip install accelerate

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import pandas as pd
from datasets import Dataset
import pickle
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM
import torch
import numpy as np
import torch.nn as nn


import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
seed = 42
pl.seed_everything(seed)


import warnings
warnings.filterwarnings("ignore")

from evaluate import load
sari = load("sari")

from readability import Readability
from tqdm import tqdm

INFO:lightning_fabric.utilities.seed:Seed set to 42


In [3]:
data_location = '/content/'
#training_args = TrainingArguments("test=trainer", evaluation_strategy="epoch")#TrainingArguments(output_dir=f"{data_location}training_args")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
config = AutoConfig.from_pretrained(model_name,
    max_new_tokens=1024
)

In [4]:
# define new class called training_arguements
class TrainingArguments:
    def __init__(self):
        self.output_dir = "./output"
        self.evaluation_strategy = "epoch"
        self.batch_size = 32
        self.adam_beta1 = 0.9
        self.adam_beta2 = 0.999
        self.adam_epsilon = 1e-8
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.lr_scheduler_type = "linear"
        self.max_grad_norm = 1.0
        self.max_steps = -1
        self.num_train_epochs = 3
        self.seed = 42
        self.warmup_steps = 0
        self.weight_decay = 0.0
        # self.logging_dir = "./logs"
        # self.logging_first_step = False
        # self.logging_steps = 500
        # self.save_steps = 500
        # self.save_total_limit = 1

    def __str__(self):
        print("Training Arguments / Hyperparameters:")
        print("---------------------------------")
        for key, value in self.__dict__.items():
            print(f"| {key}: {value}")
        return "--------------------------------"

training_args = TrainingArguments()

In [5]:
print(training_args)
print(training_args.batch_size)

Training Arguments / Hyperparameters:
---------------------------------
| output_dir: ./output
| evaluation_strategy: epoch
| batch_size: 32
| adam_beta1: 0.9
| adam_beta2: 0.999
| adam_epsilon: 1e-08
| gradient_accumulation_steps: 1
| learning_rate: 5e-05
| lr_scheduler_type: linear
| max_grad_norm: 1.0
| max_steps: -1
| num_train_epochs: 3
| seed: 42
| warmup_steps: 0
| weight_decay: 0.0
--------------------------------
32


In [6]:
# grade_ratio = pd.read_csv(f'{data_location}grade_ratio_wiki_train.csv')
# # source texts
# with open(f'{data_location}wiki_train.src', 'r', encoding='utf-8') as f:
#     train_src = f.readlines()
# train_src = pd.DataFrame(train_src, columns=['source'])
# # target texts
# with open(f'{data_location}wiki_train.tgt', 'r', encoding='utf-8') as f:
#     train_tgt = f.readlines()
# train_tgt = pd.DataFrame(train_tgt, columns=['target'])
# train_texts = pd.concat([train_src, grade_ratio['abs_src_FKGL_Grade'], train_tgt, grade_ratio['abs_tgt_FKGL_Grade']], axis=1)
# train_texts.rename(columns={'abs_src_FKGL_Grade': 'source_grade', 'abs_tgt_FKGL_Grade': 'target_grade'}, inplace=True)
# train_texts['souce'] = train_texts['source'].replace(r'\n','', regex=True)
# train_texts['target'] = train_texts['target'].replace(r'\n','', regex=True)

In [7]:
train_texts = pd.read_pickle(f'{data_location}train_texts.pkl')
train_texts.iloc[0]['source']

'Heinrich Luitpold Himmler (7 October 1900 - 23 May 1945) was Chief of the German Police and Minister of the Interior.\n'

In [8]:
grade_groups = train_texts.groupby(['target_grade'])
grade_groups.get_group(0)

,source,source_grade,target,target_grade
5,"Though founded in 1887, under Jack Hyles' lead...",13,Dr. Jack Hyles\n,0
121,"On January 27, 2008, at the NHL All-Star Game ...",7,Records\n,0
130,Gone the times when nations battled for this' ...,8,gone the days when strife and discord.\n,0
152,May 17 & ndash; The conflict between Toyotomi ...,10,Ghent falls to the Spanish.\n,0
172,Some subjects that are discussed have criminal...,11,(see and).\n,0
...,...,...,...,...
216813,"Dubnium (,) is a chemical element with the sym...",10,It has the symbol Db.\n,0
216828,WWE Hell in a Cell is a professional wrestling...,13,Hell\n,0
216833,He died of a heart attack in 1968 and was hono...,8,He died from a heart attack in 1968.\n,0
216839,"In English, the name is sometimes spelled Bela...",10,-)\n,0


In [9]:
# create custom dataset where each grade group is a separate dataset, including source, target, and target grade
datasets = {}
for i, (grade, group) in enumerate(grade_groups):
    datasets[i] = Dataset.from_pandas(group[['source', 'target', 'target_grade']]).train_test_split(test_size=0.2)

datasets[6]

DatasetDict({
    train: Dataset({
        features: ['source', 'target', 'target_grade', '__index_level_0__'],
        num_rows: 20313
    })
    test: Dataset({
        features: ['source', 'target', 'target_grade', '__index_level_0__'],
        num_rows: 5079
    })
})

In [10]:
datasets[6]['train']

Dataset({
    features: ['source', 'target', 'target_grade', '__index_level_0__'],
    num_rows: 20313
})

In [11]:
def tokenize_function(examples):
    return tokenizer(text=examples["source"], text_target=examples['target'], padding=True, max_length=128, return_tensors="pt")

tokenized_dataset_6 = datasets[6].map(tokenize_function, batched=True, batch_size=training_args.batch_size,
                                      remove_columns=['source', 'target', '__index_level_0__'])
tokenized_dataset_6

Map:   0%|          | 0/20313 [00:00<?, ? examples/s]

Map:   0%|          | 0/5079 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['target_grade', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 20313
    })
    test: Dataset({
        features: ['target_grade', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5079
    })
})

In [12]:
print(len(tokenized_dataset_6['train'][100]['input_ids']))
print(len(tokenized_dataset_6['train'][100]['attention_mask']))
print(len(tokenized_dataset_6['train'][100]['labels']))

63
63
38


In [13]:
longest_source = 0
source = ''
longest_target = 0
target = ''
for dataset in ['train', 'test']:
  for example in tokenized_dataset_6[dataset]:
      source_len = len(example['input_ids'])
      target_len = len(example['labels'])
      if source_len > longest_source:
          longest_source = source_len
          source = example['input_ids']
      if target_len > longest_target:
          longest_target = target_len
          target = example['labels']
print("longest_source:", longest_source)
print(source)
print("longest_target:", longest_target)
print(target)

longest_source: 98
[50, 945, 12, 293, 12, 33, 10924, 318, 257, 1316, 68, 287, 262, 17454, 12, 2934, 12, 34, 6081, 271, 5011, 287, 262, 18687, 12, 47, 292, 12, 2934, 12, 34, 6081, 271, 3814, 286, 4881, 13, 198, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]
longest_target: 85
[32, 282, 268, 357, 31186, 8918, 8, 318, 257, 3240, 287, 262, 2679, 1181, 286, 7772, 268, 12, 54, 25151, 926, 1491, 70, 13, 198, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, padding="max_length", max_length=longest_source, label_pad_token_id=tokenizer.pad_token_id)

train_data_loader = torch.utils.data.DataLoader(tokenized_dataset_6['train'], batch_size=training_args.batch_size, shuffle=True, collate_fn=data_collator)
eval_data_loader = torch.utils.data.DataLoader(tokenized_dataset_6['test'], batch_size=training_args.batch_size, shuffle=False, collate_fn=data_collator)
dataloaders = {'train': train_data_loader, 'eval': eval_data_loader}

In [15]:
for batch in train_data_loader:
    print(batch['input_ids'].shape)
    print(batch['attention_mask'].shape)
    print(batch['labels'].shape)
    print(batch['target_grade'].shape)
    break

torch.Size([32, 98])
torch.Size([32, 98])
torch.Size([32, 98])
torch.Size([32])


In [16]:
def compute_metrics(prediction):

    source_ids, pred_ids, labels_ids = prediction
    sources = []
    labels = []
    predictions = []

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    source_str = tokenizer.batch_decode(source_ids, skip_special_tokens=True)
    sources.append(''.join(source_str))
    labels.append([''.join(label_str)])
    predictions.append(''.join(pred_str))
    print(sources)
    print(predictions)
    print(labels)


    return sari.compute(sources=sources, predictions=predictions, references=labels)

In [17]:
compute_metrics([tokenized_dataset_6['train'][1]['input_ids'], tokenized_dataset_6['train'][1]['labels'], tokenized_dataset_6['train'][1]['labels']])

["During his playing career he played for Cowdenbeath and Heart of Midlothian. He won 16 caps for Scotland and was part of his country's 1990 World Cup squad in Italy.\n"]
['During his playing career he played for Cowdenbeath F.C., Heart of Midlothian and won 16 caps for the Scotland national team.\n']
[['During his playing career he played for Cowdenbeath F.C., Heart of Midlothian and won 16 caps for the Scotland national team.\n']]


{'sari': 97.43589743589745}

In [18]:
# input_text = '''
# Quantum mechanics is a fundamental theory in physics that provides a description of the physical properties
# of nature at the scale of atoms and subatomic particles. It is the foundation of all quantum physics including
# quantum chemistry, quantum field theory, quantum technology, and quantum information science
# '''


# trained_model = AutoModelForCausalLM.from_pretrained('./models/gpt2_wikilarge_6')
# inputs = tokenizer([input_text], padding='max_length',
#                        max_length=60, truncation=True, return_tensors='pt')

# trained_model.config.decoder_start_token_id = tokenizer.cls_token_id
# trained_model.config.eos_token_id = tokenizer.sep_token_id
# trained_model.config.pad_token_id = tokenizer.pad_token_id
# #trained_model.config.vocab_size = model.config.encoder.vocab_size

# output = trained_model.generate(inputs['input_ids'],
#                         max_length=100,
#                         min_length=30,
#                         num_beams=4,
#                         length_penalty=0.8,
#                         temperature=0.5,
#                         early_stopping=True,
#                         top_k=50,
#                         do_sample=False)

# text = tokenizer.batch_decode(output, skip_special_tokens=True)
# print(input_text)
# print(text[0])

In [19]:
# !python -m nltk.downloader punkt
# r = Readability(text[0]*3)

# r.flesch_kincaid().score

In [20]:
class GPT2forSeq2Seq(pl.LightningModule):
    def __init__(self, model_name, training_args):
        super(GPT2forSeq2Seq, self).__init__()
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.training_args = training_args
        # self.save_hyperparameters()
        self.loss = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        # print("input shape:", input_ids.shape)
        # print("attn shape:", attention_mask.shape)
        # print("labels shape:", labels.shape)
        return self.model(input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # target_grade = batch['target_grade']
        outputs = self(input_ids, attention_mask, labels=labels)
        print("training_step output:", outputs)
        loss = outputs.loss
        return loss

    def validation_step(self, batch):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # target_grade = batch['target_grade']
        outputs = self(input_ids, attention_mask, labels=labels)
        print("validation_step output:", outputs)
        loss = outputs.loss
        return loss

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x for x in outputs]).mean()
        self.log('train_loss', avg_loss)

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x for x in outputs]).mean()
        self.log('val_loss', avg_loss)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.training_args.learning_rate)
        return optimizer

In [24]:
def train_test(model, dataloader, optimizer, training):
    """
    Performs a single epoch of training, validation, or testing on the given model using the specified DataLoader.
    This function adapts its behavior based on the 'training' parameter to correctly handle the model's state and
    perform necessary operations such as backpropagation and optimizer updates during training.

    Parameters:
        model (torch.nn.Module): The neural network model to be trained, validated, or tested.
        dataloader (DataLoader): A DataLoader providing batches of data (features and labels) for processing.
        optimizer (torch.optim.Optimizer): The optimizer (AdamW) to use for updating model parameters during training.
        pos_weight (torch.Tensor): A tensor specifying the weight for the positive class to handle class imbalance.
        training (str): A string specifying the mode of operation. Must be 'train', 'validation', or 'test'.

    Returns:
        None if training.
        Cumulative loss (float) if validation.
        A tuple (label_list, prediction_list) containing lists of true labels and predicted labels for
        each sample if testing.
    """
    # BCEWithLogitsLoss combines sigmoid with BCELoss for better stability, and handles class imbalance via pos_weight

    if training == "train":
        model.train()
    elif training == "validation":
        model.eval()
    elif training == "test":
        model.eval()
    else:
        raise ValueError("training argument must be either 'train', 'validation' or 'test'")

    cumulative_loss = 0
    prediction_list = [] # store predictions accross folds for calculating accuracy and f1
    label_list = [] # store labels accross folds for calculating accuracy and f1
    loss_function = torch.nn.CrossEntropyLoss()

    for sample in tqdm(dataloader): # iterate over batches in the DataLoader
        sample.to(device)
        input, attention_mask, labels = sample["input_ids"], sample["attention_mask"], sample['labels']
        output = model(input, attention_mask, labels) # forward pass
        loss_value = output.loss
        cumulative_loss += loss_value.item()

        if training == "train":
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()

        #predictions = [round(x) for x in sigmoid(output).to('cpu').detach().squeeze(1).numpy().tolist()] # gets {0,1} predictions from 1d logits
        target_labels = labels.to('cpu').detach().numpy()
        #prediction_list.extend(predictions)
        label_list.extend(target_labels)
        print(loss_value.item())

    if training == "train":
        print("mean training loss:", cumulative_loss/len(dataloader))
        return cumulative_loss
    elif training == "validation":
        print("mean validation loss:", cumulative_loss/len(dataloader))
        return cumulative_loss
    elif training == "test":
        return label_list, prediction_list
    else:
        raise ValueError("Ya Done Fuck'd up, son!")

In [25]:
# Training sample
def evaluate(data, parameters):
    """
    Evaluates neural model's performance on a given task using specified parameters.
    The function preprocesses the data, splits it according to the task, initializes a TuneableModel,
    and trains it. It then evaluates the model on a test set and returns performance metrics.

    The function asserts the task to be one of the predefined tasks and initializes the model based on
    the provided parameters. It supports dynamic pos_weight handling and uses early stopping based on
    validation loss to prevent overfitting.

    Parameters:
        data (pd.DataFrame): The dataset to evaluate the model on.
        parameters (dict): A dictionary containing model hyperparameters and training settings. Expected
            keys include "pos_weight", "batch_size", "alpha", "hidden_size", "dropout", "n_hidden",
            "learning_rate", "beta_1", and "beta_2".
        task (int): An integer indicating the task type. Valid values are 0, 1, 2, and 3, each representing
            a different way of splitting the data for training and testing:
                0 - Known subjects and items with k-fold cross-validation.
                1 - Known subjects and items with leave-one-out cross-validation.
                2 - Held-out subjects, known items.
                3 - Held-out items, known subjects.

    Returns:
        tuple: A tuple containing the accuracy score, F1 score, and confusion matrix of the model evaluated
            on a given test set.
    """

    max_epochs = 1000

    predictions = []
    labels = []
    torch.manual_seed(seed)
    model = GPT2forSeq2Seq(model_name=model_name, training_args=training_args)
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=parameters.learning_rate,
                                  betas=(parameters.adam_beta1, parameters.adam_beta2), weight_decay=parameters.adam_epsilon)

    max_patience = 2
    last_loss = 1000000
    PATH = f"./models/model_{i}.pt"
    for epoch in range(max_epochs):
        # training
        train_test(model, train_data_loader, optimizer, training="train")
        # validation at end of epoch
        with torch.no_grad():
            validation_loss = train_test(model, eval_data_loader, optimizer, training="validation")

        if validation_loss < last_loss:
            last_loss = validation_loss
            current_patience = 0
        else:
            if current_patience == 0:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': last_loss,
                    }, PATH)
            current_patience += 1
        if current_patience == max_patience:
            break

    # # Testing once patience is reached
    # torch.manual_seed(seed)
    # model = TuneableModel(input_size, parameters["hidden_size"], parameters["dropout"], parameters["n_hidden"])
    # model.to(device)
    # optimizer = torch.optim.AdamW(model.parameters(), lr=parameters["learning_rate"], betas=(0.99, 0.99), weight_decay=1e-4)
    # checkpoint = torch.load(PATH)
    # model.load_state_dict(checkpoint['model_state_dict'])
    # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # with torch.no_grad():
    #     prediction_list, label_list = train_test(model, test_dataloader, optimizer, training="test")
    # predictions.extend(prediction_list)
    # labels.extend(label_list)

    return #compute_metrics() # insert sari ids


In [26]:
evaluate(tokenized_dataset_6, training_args)

  0%|          | 1/635 [00:00<09:37,  1.10it/s]

10.070459365844727


  0%|          | 2/635 [00:01<09:18,  1.13it/s]

6.596116542816162


  0%|          | 3/635 [00:02<09:12,  1.14it/s]

...

0.9661607146263123


  3%|▎         | 19/635 [00:17<09:23,  1.09it/s]

1.4026761054992676


  3%|▎         | 19/635 [00:18<09:52,  1.04it/s]


KeyboardInterrupt: 